In [2]:
%reload_ext autoreload
%autoreload 2
from termcolor import colored
import dotenv
import sys
import dspy
import os

from tqdm.auto import tqdm

sys.path.append('../pipeline_v2/')
import main 
dotenv.load_dotenv('../.env')

from utils import print_header

import pandas as pd

def print_final_result(statement, verdict, confidence, reasoning, gold_verdict=None):
    # Print final result
    print("\nFinal Fact-Check Result:")
    print_header(f"Statement: {colored(statement, 'white')}", level=1)
    print_header(f"Overall Verdict: {colored(verdict, 'green')}", level=1)
    print_header(f"Overall Confidence: {colored(str(confidence), 'yellow')}", level=1)
    print_header(f"Overall Reasoning: {colored(reasoning, 'cyan')}", level=1)
    if gold_verdict: print_header(f"Gold Verdict: {colored(gold_verdict, 'green')}", level=1)

In [3]:
### Load data
if os.path.exists('results_v2.pkl'):
    df = pd.read_pickle('results_v2.pkl')
else: 
    df = pd.read_csv('../data/pilot_updated_v2.csv')

    # # Drop unneeded columns
    df.drop(columns=['Assignee', 'questions to verify the statement', 'Gold Label', 'factcheck_date'], inplace=True)

    # Reformat dates
    df['statement_date'] = pd.to_datetime(df['statement_date']).dt.strftime("%B %d, %Y")

df

,verdict,statement_originator,statement,statement_date,context,factchecker,factcheck_analysis_link,Reasoning
0,MOSTLY TRUE,Tony Evers,“After 30 years of Wisconsin’s checking accoun...,"December 28, 2024",NaN,PolitiFact,https://www.politifact.com/factchecks/2025/jan...,State budget season is approaching in Wisconsi...
1,TRUE,Sara Jacobs,Government shutdowns in 2013 and 2018 “cost ou...,"December 18, 2024",NaN,PolitiFact,https://www.politifact.com/factchecks/2024/dec...,As Congress headed toward a potential pre-Chri...
2,FALSE,Mike Johnson,About 1% of federal employees are “actually wo...,"December 05, 2024",NaN,PolitiFact,https://www.politifact.com/factchecks/2024/dec...,As high-profile businesspeople Elon Musk and V...
3,HALF TRUE,Roy Cooper,North Carolina Republicans “took money out of ...,"December 03, 2024",NaN,PolitiFact,https://www.politifact.com/factchecks/2025/jan...,North Carolina legislators in December enacted...
4,TRUE,Jay Rothman,“The Universities of Wisconsin are 43rd out of...,"December 01, 2024",NaN,PolitiFact,https://www.politifact.com/factchecks/2024/dec...,State funding for the University of Wisconsin ...
...,...,...,...,...,...,...,...,...
145,FALSE,Fox News host Bret Baier,"""Former U.S. President Donald Trump is the fir...","January 23, 2024",News,Snopes,https://www.snopes.com/fact-check/trump-new-ha...,NaN
146,HALF TRUE,Joe Biden,"""Look, folks, you know how many billionaires w...","January 18, 2024",Speech,Washington Post,https://www.washingtonpost.com/politics/2024/0...,NaN
147,MOSTLY TRUE,Ron DeSantis,“[Nikki Haley] spent 100% of her money attacki...,"January 16, 2024",Speech,CNN,https://www.cnn.com/2024/01/16/politics/fact-c...,NaN
148,FALSE,Victor Shi,"""Former U.S. President Donald Trump's margin o...","January 15, 2024",Post,Snopes,https://www.snopes.com/fact-check/donald-trump...,NaN


In [6]:
# Set custom constants for whole pipeline
main.VERBOSE = True # Print intermediate results
# main.VERDICTS=["Supported", "Refuted", "Not Enough Evidence", "Conflicting Evidence/Cherry-picking"]

# Initialize DSPy
# lm = dspy.LM('gemini/gemini-1.5-pro', api_key=os.getenv('GOOGLE_GEMINI_API_KEY'), cache=False)
lm = dspy.LM('ollama_chat/mistral', api_base='http://localhost:11434', api_key='', cache=False)
# lm = dspy.LM('ollama_chat/llama3.1:8b', api_base='http://localhost:11434', api_key='', cache=False)
# lm = dspy.LM('ollama_chat/deepseek-r1:7b', api_base='http://localhost:11434', api_key='', cache=False)
dspy.settings.configure(lm=lm, temperature=0.3)

pipeline = main.FactCheckPipeline(
    search_provider=main.SearchProvider(provider="duckduckgo"),
    model_name=lm,
    embedding_model=main.EMBEDDING_MODEL,
    retriever_k=5
)

In [8]:
# Test with a single statement from dataset
index = -3
statement = df.iloc[index]['statement']
statement_originator = df.iloc[index]['statement_originator']
statement_date = df.iloc[index]['statement_date']
gold_verdict = df.iloc[index]['verdict']

verdict, confidence, reasoning, claims = pipeline.fact_check(
    statement=f"According to {statement_originator} on {statement_date}, {statement}", 
    # statement=statement, 
    # context=f"Statement Originator: {statement_originator}, Date Claim Was Made: {statement_date}"
)
print_final_result(statement, verdict, confidence, reasoning, gold_verdict)

===== Starting Fact Check Pipeline =====
 Original Statement: According to Ron DeSantis on January 16, 2024, “[Nikki Haley] spent 100% of her money attacking me, and not one red cent attacking Donald Trump,” DeSantis claimed. “And I faced almost $50 million in total. I got in second and she did not.” 
  ===== Atomic Claim Extraction =====
   Extracted Claims (2):  
     1. Nikki Haley spent all her money attacking me (Ron DeSantis) instead of Donald Trump. 
     2. I faced almost $50 million in total. I got in second and she did not. 
    ===== Question Generation [1/2] =====
       Decomposed Components (Questions + Search Queries) (3): 
         1. Question: What was the distribution of campaign donations made by Nikki Haley on January 16, 2024? 
            Search Queries: ['Nikki Haley campaign donations 2024-01-16', 'Nikki Haley campaign expenditures 2024-01-16'] 
         2. Question: Were the majority of Nikki Haley's campaign donations on January 16, 2024 directed towards attac

        ===== Web Search for Query [2/2] =====
         Query: Nikki Haley campaign expenditures 2024-01-16 
         Retrieved 10 Sources: 
           1. Expenditures Breakdown, Nikki Haley, 2024 cycle • OpenSecrets 
           URL: https://www.opensecrets.org/2024-presidential-race/nikki-haley/expenditures?id=N00052386 
           Excerpt: This page highlights the expenditures breakdown for Nikki Haley for the 2024 presidential election cycle. For more information, click through to the page. ... Search campaign expenditures to explore how candidates and political groups are spending donor money. ... 16: HDS Marketing: $2,289,636: 21: Political Travel Partners: $2,145,304: 45 ... 
           2. Expenditures Breakdown, Nikki Haley, 2024 cycle • OpenSecrets 
           URL: https://www.opensecrets.org/2024-presidential-race/nikki-haley/expenditures?id=N00052386THANKS, 
           Excerpt: This page highlights the expenditures breakdown for Nikki Haley for the 2024 presidential election 

      ===== Synthesizing Answer [1/3] =====
         Question: What was the distribution of campaign donations made by Nikki Haley on January 16, 2024? 
         Search Queries: ['Nikki Haley campaign donations 2024-01-16', 'Nikki Haley campaign expenditures 2024-01-16'] 


         Answer: The information provided suggests that Nikki Haley received campaign donations totaling $16.5 million in January 2024, with grassroots supporters contributing $11.7 million. However, the exact distribution of these donations on January 16, 2024, is not specified. 

Reasoning: The search queries were designed to find information about campaign donations made by Nikki Haley on January 16, 2024. However, it is important to note that campaign donations are typically reported in monthly totals rather than daily or individual transactions. The documents found provide evidence that Nikki Haley raised $16.5 million in January 2024, which includes grassroots supporters' contributions of $11.7 million. While the exact distribution of these donations on January 16, 2024, is not specified, it can be inferred that a portion of this total was raised on that day. 
      ===== Question Answering for Component [2/3] =====
         Question: Were the majority of Nikki Haley's campaign d

        ===== Web Search for Query [2/2] =====
         Query: Nikki Haley campaign spending against Ron DeSantis 2024-01-16 
         Retrieved 10 Sources: 
           1. Nikki Haley bashes Ron DeSantis over campaign spending: 'How can you ... 
           URL: https://floridapolitics.com/archives/652495-nikki-haley-bashes-ron-desantis-over-campaign-spending-how-can-you-manage-a-country/ 
           Excerpt: Just look at their spending on the campaign trail, Nikki Haley said Wednesday in a debate stage dig at Gov. Ron DeSantis. "The best way to tell about a candidate is how they run their campaign ... 
           2. What campaign ad spending says about the 2024 election ahead of New ... 
           URL: https://www.npr.org/2024/01/19/1225550399/campaign-ad-spending-trump-desantis-haley-biden 
           Excerpt: Nikki Haley is acting like Ron DeSantis isn't even in the race anymore — and DeSantis' campaign isn't spending on ads at all. Here's a breakdown of their closing arguments befo

...

      ===== Synthesizing Answer [1/2] =====
         Question: What was the total amount of money spent by Ron DeSantis on January 16, 2024? 
         Search Queries: ['Ron DeSantis campaign expenses 2024-01-16'] 
         Answer: The total amount spent by Ron DeSantis on January 16, 2024 was $551,925 [1]. 

Reasoning: To find the total amount of money spent by Ron DeSantis on January 16, 2024, I searched for relevant documents containing information about his campaign expenses on that specific date. The documents provided contain details about his fundraising and expenditures during the 2024 election cycle. By examining the expenditure breakdown document, I found a record of transactions made by DeSantis' campaign on January 16, 2024. 
         Citations:  
           [1] Tyson Group: $551,925 
           Source: Expenditures Breakdown, Ron DeSantis, 2024 cycle • OpenSecrets (https://www.opensecrets.org/2024-presidential-race/ron-desantis/expenditures?id=N00034746) 
      ===== Questi

        ===== Web Search for Query [2/2] =====
         Query: Nikki Haley election results 2024-01-16 
         Retrieved 10 Sources: 
           1. Nikki Haley wins DC Republican primary for first 2024 win - The State 
           URL: https://www.thestate.com/news/politics-government/article286217330.html 
           Excerpt: Nikki Haley has won the District of Columbia's Republican primary for her first ... Democrat Joe Biden won the district in the 2020 general election with 92% of the vote. ... Updated January 16 ... 
           2. Landslide Trump win tests Haley, DeSantis heading into New Hampshire ... 
           URL: https://www.washingtonpost.com/politics/2024/01/16/trump-iowa-new-hampshire-haley/ 
           Excerpt: Updated January 16, 2024. 10 min. Election results are shown in the press room at Republican presidential candidate Nikki Haley's caucus night party in Des Moines on Monday. (Melina Mara/The ... 
           3. Haley attracts more than 150K votes in Pennsylvania G

      ===== Synthesizing Answer [2/2] =====
         Question: What was the position of Ron DeSantis compared to Nikki Haley on January 16, 2024? 
         Search Queries: ['Ron DeSantis election results 2024-01-16', 'Nikki Haley election results 2024-01-16'] 


Attempt 1 failed: 1 validation error for dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value='Not a candidate', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/dict_type
         Question: What was the position of Ron DeSantis compared to Nikki Haley on January 16, 2024? 
         Search Queries: ['Ron DeSantis election results 2024-01-16', 'Nikki Haley election results 2024-01-16'] 
Attempt 2 failed: 1 validation error for dict[any,any]
  Input should be a valid dictionary [type=dict_type, input_value='Not a candidate', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/dict_type
         Question: What was the position of Ron DeSantis compared to Nikki Haley on January 16, 2024? 
         Search Queries: ['Ron DeSantis election results 2024-01-16', 'Nikki Haley election results 2024-01-16'] 
Attempt 3 failed: 1 validation error for dict[any,any]
  Input should be a valid dictionary [typ

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
model = 'gemini-1.5-pro'
num_trials = 3

# If column doesn't exist, create it
if f'{model}_results' not in df.columns: df[f'{model}_results'] = None
df[f'{model}_results'] = df[f'{model}_results'].astype(object)

for index in tqdm(range(len(df))):
    # If results already exist, skip if num_trials is reached
    if df.loc[index, f'{model}_results'] is not None: 
        if len(df.loc[index, f'{model}_results']) == num_trials:
            continue
        else:
            results = df.loc[index, f'{model}_results']
    else: 
        results = []

    for trial_i in tqdm(range(num_trials), leave=False):
        statement = df.iloc[index]['statement']
        statement_originator = df.iloc[index]['statement_originator']
        statement_date = df.iloc[index]['statement_date']
        gold_verdict = df.iloc[index]['verdict']

        verdict, confidence, reasoning, claims = pipeline.fact_check(
            statement=statement, 
            context=f"Statement Originator: {statement_originator}, Date Claim Was Made: {statement_date}"
        )
        results.append({
            'verdict': verdict,
            'confidence': confidence,
            'reasoning': reasoning,
            'claims': claims
        })

        print_final_result(statement, verdict, confidence, reasoning, gold_verdict)

    # Update the dataframe
    df.at[index, f'{model}_results'] = results

In [11]:
df.to_pickle('results_v2.pkl')

## Analysis

In [19]:
import pandas as pd

df = pd.read_pickle('results_v2_gemini-pro.pkl')

# For benchmarking, we want to compare the performance of the model to the gold label
# Since we generate multiple verdicts, we want to see if the model's verdict is in the top 3
# We can do this by creating a new column in the dataframe that checks if the model's verdict is in the top 3
# We can then group by the gold label and see the percentage of times the model's verdict is in the top 3

# df['model_verdict_in_top_3'] = df.apply(lambda row: row['model_verdict'] in row['gold_verdict_top_3'], axis=1)
df

,verdict,statement_originator,statement,statement_date,context,factchecker,factcheck_date,factcheck_analysis_link,gemini_results
0,FALSE,Instagram posts,“The National Guard in the HISTORY of its life...,"April 02, 2024",Social Media,Politifact,4/8/2024,https://www.politifact.com/factchecks/2024/apr...,"[{'verdict': 'MOSTLY FALSE', 'confidence': 0.8..."
1,PANTS ON FIRE,ROBERT F. Kennedy Jr.,"""On Jan. 6, 2021, U.S. Capitol 'protestors car...","April 05, 2024",Written Copy on Website,Politifact,04/05/2024,NaN,"[{'verdict': 'FALSE', 'confidence': 1.0, 'reas..."
2,FALSE,Threads Post,"""Not even one rocket (from Iran) hit Israel.""","April 14, 2024",Social Media,Politifact,4/15/2024,https://www.politifact.com/factchecks/2024/apr...,"[{'verdict': 'FALSE', 'confidence': 0.9, 'reas..."
3,FALSE,Instagram Post,"""326,000 migrants were flown to Florida with t...","April 04, 2024",Social Media,Politifact,4/12/2024,https://www.politifact.com/factchecks/2024/apr...,"[{'verdict': 'MOSTLY FALSE', 'confidence': 0.9..."
4,FALSE,Donald Trump,"""Crime is down in Venezuela by 67% because the...","April 02, 2024",Speech,Politifact,4/10/2024,https://www.politifact.com/factchecks/2024/apr...,"[{'verdict': 'FALSE', 'confidence': 1.0, 'reas..."
...,...,...,...,...,...,...,...,...,...
78,FALSE,Nicole Shanahan,"""I will be the the youngest vice president in ...","March 26, 2024",Speech,factcheck.org,3/27/2024,https://factcheck.org/2024/03/factchecking-rfk...,None
79,FALSE,Nicole Shanahan,"""Pharmaceutical medicine” was one of “three ma...","March 26, 2024",Speech,factcheck.org,3/27/2024,factcheck.org/2024/03/factchecking-rfk-jr-s-v-...,None
80,FALSE,Donald Trump,"""This year, the typical family’s tax bill is t...","April 15, 2024",Truth Social,factcheck.org,4/17/2024,https://www.factcheck.org/2024/04/trumps-unfou...,None
81,MOSTLY FALSE,Robert F. Kennedy Jr.,“Those policies that both of them engineered t...,"March 26, 2024",Speech,factcheck.org,3/27/2024,https://factcheck.org/2024/03/factchecking-rfk...,None


In [41]:
import pandas as pd

df_pro = pd.read_pickle('results_v2_gemini-pro.pkl')
df_flash = pd.read_pickle('results_v2_gemini.pkl')

# Extract the verdicts from the gemini_results column
df_pro['pred_verdicts_gemini-pro'] = df_pro['gemini_results'].apply(lambda x: [result['verdict'] for result in x] if x else None)
df_flash['pred_verdicts_gemini-flash'] = df_flash['gemini_results'].apply(lambda x: [result['verdict'] for result in x] if x else None)

# Merge the two dataframes on the statement column 
df_merged = pd.merge(df_pro, df_flash, on=['statement', 'verdict'], suffixes=('_pro', '_flash'))
df_merged[['statement', 'verdict', 'pred_verdicts_gemini-pro', 'pred_verdicts_gemini-flash']].head(20)

,statement,verdict,pred_verdicts_gemini-pro,pred_verdicts_gemini-flash
0,“The National Guard in the HISTORY of its life...,FALSE,"[MOSTLY FALSE, MOSTLY TRUE, MOSTLY TRUE]","[UNVERIFIABLE, UNVERIFIABLE, UNVERIFIABLE]"
1,"""On Jan. 6, 2021, U.S. Capitol 'protestors car...",PANTS ON FIRE,"[FALSE, FALSE, FALSE]","[FALSE, FALSE, FALSE]"
2,"""Not even one rocket (from Iran) hit Israel.""",FALSE,"[FALSE, FALSE, FALSE]","[FALSE, FALSE, FALSE]"
3,"""326,000 migrants were flown to Florida with t...",FALSE,"[MOSTLY FALSE, FALSE, MOSTLY TRUE]","[FALSE, FALSE, FALSE]"
4,"""Crime is down in Venezuela by 67% because the...",FALSE,"[FALSE, MOSTLY FALSE, MOSTLY FALSE]","[MOSTLY FALSE, MOSTLY FALSE, FALSE]"
5,"""In New York, there are no barriers to law enf...",MOSTLY FALSE,"[HALF TRUE, HALF TRUE, UNVERIFIABLE]",[UNVERIFIABLE]
6,"""Speaking of semiconductor industry jobs, ""Kno...",MOSTLY FALSE,"[HALF TRUE, HALF TRUE, MOSTLY TRUE]",None
7,"""Starting in 2025 ""no matter what your total b...",MOSTLY TRUE,"[MOSTLY TRUE, MOSTLY TRUE, HALF TRUE]",None
8,“Tens of thousands of auto jobs were lost nati...,HALF TRUE,"[MOSTLY FALSE, HALF TRUE, MOSTLY FALSE]",None
9,"""The current Congress is “the least productive...",MOSTLY TRUE,"[MOSTLY TRUE, MOSTLY TRUE, UNVERIFIABLE]",None
